In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

**Simple_conv_neuralNetwok-pytorch**

**From_scratch**








In [0]:
#DownLoad the Dataset if not  downloaded in LocalComputer,
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

In [0]:
#hyperparameters
num_classes =10 
batch_size = 100
learning_rate = 0.0001
num_epochs = 5



In [0]:
# Device configuration for GPU's if present on the Machine
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [0]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [0]:
# Convolutional neural network
# Two convolutional layers


class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [0]:
model = ConvNet(num_classes).to(device)


In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


In [31]:
#This is Training Step. This may take Time if GPU is not Available
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 1.0617
Epoch [1/5], Step [200/600], Loss: 0.5260
Epoch [1/5], Step [300/600], Loss: 0.3855
Epoch [1/5], Step [400/600], Loss: 0.2331
Epoch [1/5], Step [500/600], Loss: 0.2806
Epoch [1/5], Step [600/600], Loss: 0.2561
Epoch [2/5], Step [100/600], Loss: 0.1571
Epoch [2/5], Step [200/600], Loss: 0.1550
Epoch [2/5], Step [300/600], Loss: 0.1935
Epoch [2/5], Step [400/600], Loss: 0.1484
Epoch [2/5], Step [500/600], Loss: 0.0992
Epoch [2/5], Step [600/600], Loss: 0.0783
Epoch [3/5], Step [100/600], Loss: 0.1258
Epoch [3/5], Step [200/600], Loss: 0.0935
Epoch [3/5], Step [300/600], Loss: 0.1090
Epoch [3/5], Step [400/600], Loss: 0.0870
Epoch [3/5], Step [500/600], Loss: 0.1036
Epoch [3/5], Step [600/600], Loss: 0.0956
Epoch [4/5], Step [100/600], Loss: 0.0466
Epoch [4/5], Step [200/600], Loss: 0.0660
Epoch [4/5], Step [300/600], Loss: 0.1141
Epoch [4/5], Step [400/600], Loss: 0.0683
Epoch [4/5], Step [500/600], Loss: 0.0690
Epoch [4/5], Step [600/600], Loss:

In [32]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 98.3 %


In [0]:
# Save the model checkpoint

torch.save(model.state_dict(), 'model.ckpt')